#### Generator

> ![SRGAN Generator](https://github.com/https-deeplearning-ai/GANs-Public/blob/master/SRGAN-Generator.png?raw=true)


In [ ]:
class Shuffle_Block(nn.Module):
  def __init__(self, channels = 64):
    super().__init__()
    self.block = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels =4*channels, kernel_size = 3, stride = 1, padding = 1),
        nn.PixelShuffle(upscale_factor = 2),
        nn.PReLU()
    )

  def forward(self, x):
    return self.block(x) # [B, channels, W*2, H*2]

In [ ]:
class Residual_Block(nn.Module):
  def __init__(self, channels = 64):
    super().__init__()
    self.block = nn.Sequential(
       nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 1, padding = 1),
       nn.BatchNorm2d(num_features = channels),
       nn.PReLU(),

       nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 1, padding = 1),
       nn.BatchNorm2d(num_features = channels),
    )

  def forward(self, x):
    return x+self.block(x) # elementwise summ

In [ ]:
class Generator(nn.Module):
  def __init__(self, channels = 64, num_residual_blocks = 10, num_shuffle_blocks = 2 ):
    super().__init__()

    self.in_layer = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = channels, kernel_size = 9, stride = 1, padding = 4),
        nn.PReLU(),
    )

    self.mid_layer = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(channels)
    )

    self.end_layer = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = 3, kernel_size = 9, stride = 1, padding = 4),
        nn.Tanh()
    )
    
    self.residual_blocks = []
    for _ in range(num_residual_blocks):
      self.residual_blocks+=[Residual_Block(channels = channels)]
    
    self.shuffle_blocks = []
    for _ in range(num_shuffle_blocks):
      self.shuffle_blocks+=[Shuffle_Block(channels = channels)]

    self.residual_blocks = nn.Sequential(*self.residual_blocks)
    self.shuffle_blocks = nn.Sequential(*self.shuffle_blocks)


  def forward(self, x):
    x = self.in_layer(x)
    x = x + self.residual_blocks(x)
    x = self.shuffle_blocks(x)
    return self.end_layer(x)

#### Discriminator

### Discriminator


![SRGAN Generator](https://github.com/https-deeplearning-ai/GANs-Public/blob/master/SRGAN-Discriminator.png?raw=true)




In [ ]:
class Convolution_Blocks(nn.Module):
  def __init__(self, channels):
    super().__init__()

    self.block = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = 2*channels, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(2*channels),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(in_channels = 2*channels, out_channels = 2*channels, kernel_size = 3, stride = 2, padding = 1),
        nn.BatchNorm2d(2*channels),
        nn.LeakyReLU(0.2, inplace=True),
    )

  def forward(self, x):
      return self.block(x)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, channels = 64, num_conv_blocks = 3):
    super().__init__()

    self.int_layer = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = channels, kernel_size = 3, stride = 1),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size = 3, stride = 2),
        nn.BatchNorm2d(channels),
        nn.LeakyReLU(0.2, inplace=True)
    )

    self.end_layer = nn.Sequential(
        nn.AdaptiveAvgPool2d(2),
        nn.Flatten(),
        nn.Linear(in_features= channels*2**num_conv_blocks*2*2, out_features = 1024),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Linear(in_features = 1024, out_features = 1),
        nn.Sigmoid()
        )
    

    self.conv_blocks = []
    for i in range(num_conv_blocks):
      self.conv_blocks+=[Convolution_Blocks(channels*2**(i))]  # 64-> 128, 128->256, 256->512 if num_conv_blocks==3
    
    self.conv_blocks = nn.Sequential(*self.conv_blocks)

    
 
  def forward(self, x):
    x = self.int_layer(x)
    x = self.conv_blocks(x)
    x = self.end_layer(x)
    
    return x